In [1]:
import datetime
from tqdm.auto import tqdm
import pandas as pd
from traffic.data import opensky
from traffic.core import Traffic
from functions.data_filtering import ICAO_codes

In [2]:
# load minimum data set
df = pd.read_csv("data/go_arounds_augmented.csv", low_memory=False)
df["time"] = pd.to_datetime(df["time"])

# select London City Airport, go-arounds, and 2019-01-04
airport = ICAO_codes["heathrow"]
start = datetime.datetime(year=2019, month=1, day=1).replace(
    tzinfo=datetime.timezone.utc
)
stop = datetime.datetime(year=2019, month=2, day=1).replace(
    tzinfo=datetime.timezone.utc
)

df_selection = df.query(f"airport==@airport & has_ga & (@start <= time <= @stop)")

In [17]:
from functions.data_processing import split_flights

# iterate over flights and pull the data from OpenSky Network
flights = []
delta_time = pd.Timedelta(days=1)
for _, row in tqdm(df_selection.iterrows(), total=df_selection.shape[0]):
    # take at most 10 minutes before and 10 minutes after the landing or go-around
    start_time = row["time"] - delta_time
    stop_time = row["time"] + delta_time

    # fetch the data from OpenSky Network
    flights.append(
        opensky.history(
            start=start_time.strftime("%Y-%m-%d %H:%M:%S"),
            stop=stop_time.strftime("%Y-%m-%d %H:%M:%S"),
            callsign=row["callsign"],
            return_flight=True,
        )
    )
# The flights can be converted into a Traffic object
traffic = Traffic.from_flights(flights)
fixed_flights = []
final_list = split_flights(flights)

  0%|          | 0/75 [00:00<?, ?it/s]

In [15]:
traffic.map_leaflet()

Map(center=[48.55966298927035, 5.366837409618487], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [28]:
final_list[3].map_leaflet()

Map(center=[54.71463501621589, 21.554543801317838], controls=(ZoomControl(options=['position', 'zoom_in_text',…